In [ ]:
# Importar bibliotecas necesarias
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import files

# Cargar el archivo .txt
uploaded = files.upload()

# Leer el archivo cargado
file_name = list(uploaded.keys())[0]
with open(file_name, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
tokenizer = Tokenizer(char_level=True, filters=None)
tokenizer.fit_on_texts([text])
total_chars = len(tokenizer.word_index) + 1

# Crear secuencias para entrenamiento
seq_length = 60  # Secuencias más largas para más contexto
step = 3
input_sequences = []
output_chars = []

for i in range(0, len(text) - seq_length, step):
    input_sequences.append(text[i: i + seq_length])
    output_chars.append(text[i + seq_length])

# Convertir texto a secuencias numéricas
input_sequences = tokenizer.texts_to_sequences(input_sequences)
output_chars = tokenizer.texts_to_sequences(output_chars)

# Convertir a arrays y categorizar la salida
X = np.array(input_sequences)
y = to_categorical(np.array(output_chars) - 1, num_classes=total_chars)

# Construir el modelo LSTM con los nuevos parámetros
model = Sequential([
    Embedding(total_chars, 256, input_length=seq_length),  # Embedding más rico
    LSTM(300, return_sequences=True),  # Capa LSTM con 300 unidades
    Dropout(0.4),
    LSTM(300, return_sequences=True),
    Dropout(0.4),
    LSTM(300, return_sequences=False),  # Última capa LSTM
    Dropout(0.5),
    Dense(512, activation='relu'),  # Capa densa más robusta
    Dense(total_chars, activation='softmax')  # Salida con softmax
])

# Compilar el modelo con el optimizador Adam y un lr de 0.0005
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy')

# Callback para reducir la tasa de aprendizaje si no mejora
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

# Entrenar el modelo
model.fit(X, y, batch_size=32, epochs=100, callbacks=[reduce_lr])  # Más épocas y callback

# Guardar el modelo con el nombre solicitado
model.save('modelo_LSTM_Simple_parlamentarioAbascal.h5')

# Función para muestreo con temperatura
def sample_with_temperature(predictions, temperature=0.7):
    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions + 1e-8) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

# Función para generar texto con prefijo
def generate_text_with_prefix(prefix, max_words=120, temperature=0.7):
    result = prefix  # Usar el texto ingresado como inicio
    seed_text = prefix[-seq_length:]  # Limitar el seed a la longitud del modelo
    word_count = len(prefix.split())  # Contar las palabras iniciales

    while word_count < max_words:
        # Convertir texto a secuencia
        encoded = tokenizer.texts_to_sequences([seed_text])
        encoded = pad_sequences(encoded, maxlen=seq_length, truncating='pre')
        # Predecir el siguiente carácter
        predictions = model.predict(encoded, verbose=0)[0]
        predicted_index = sample_with_temperature(predictions, temperature)
        predicted_char = tokenizer.index_word.get(predicted_index + 1, '')

        # Añadir el carácter generado al resultado
        result += predicted_char
        seed_text = seed_text[1:] + predicted_char

        # Actualizar el contador de palabras
        word_count = len(result.split())
        if predicted_char == ' ' and word_count >= max_words:
            break

    return result.strip()

# Modo conversación mejorado
def chat_with_model():
    print("Inicia una conversación con el modelo. Escribe 'salir' para terminar.")
    while True:
        prompt = input("Tú: ")
        if prompt.lower() == 'salir':
            break
        # Generar respuesta basándose en el prompt
        response = generate_text_with_prefix(prompt, max_words=120, temperature=0.7)
        print(f"Modelo: {response}")

# Iniciar conversación
chat_with_model()


Saving intervencionesAbascal.txt to intervencionesAbascal (1).txt
Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - loss: 3.1890 - learning_rate: 5.0000e-04
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 2.9951 - learning_rate: 5.0000e-04
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 2.9630 - learning_rate: 5.0000e-04
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 2.8408 - learning_rate: 5.0000e-04
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 2.5852 - learning_rate: 5.0000e-04
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 2.4295 - learning_rate: 5.0000e-04
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 2.3490 - learning_rate: 5.0000e-04
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: 2.2927 - learning_rate: 5.0000e-04
Epoch 9/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: 2.2264 - learning_rate: 5.0000e-04
Epoch 10/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss

Inicia una conversación con el modelo. Escribe 'salir' para terminar.
Tú: Sánchez dice usted que
Modelo: Sánchez dice usted que probién de vabúlación de vabúícia—,
¿¿sabe usted, que 1s0 0o quieren je les que fueden pagar la demosgién de la jenía aquí ha aportado su provigin ilegal. y señora más grovente? y señorías, bor la demontra de sextros. su milpo… tora musticamos y le que usted no los justición mobierno.
estamábres. y usted gobla la que usted dira. 3y nos bastinamos como la dizión sucial.. ¡a suy de atroventa. que se ha hiento hicile que usted na los españoles presidente usted con los que no quieren jebrá la gisidente —y es españa, los culpables son que hijia y sus muchos estánas, más fritivas; y yo vay a demio— sobruna con la millones de españoles y u
Tú: Casado usted solo repite lo que dice
Modelo: Casado usted solo repite lo que dice y porque el porque lije el persinto de suste que usted nl que importe montrucios,
lel porque no se ha hecho, su milpúencién con las ríasticas imp